### --- Day 22: Monkey Market ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/22

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 22);
WriteLines(inputLines);

Loading puzzle file: Day22.txt
Total lines: 1635
Max line length: 8

8379519
8825018
7346587
8726021
1408335


Ok, so the simple approach is to implement the mix / prune functions and generate, but I suspect this won't be enough for stage 2 :) Probably we need to cache the results or something like that, but we'll cross that bridge when we come to it.

In [4]:
using Num = uint;

In [5]:
// To mix a value into the secret number, calculate the bitwise XOR of the given
// value and the secret number. Then, the secret number becomes the result of that
// operation. (If the secret number is 42 and you were to mix 15 into the secret
// number, the secret number would become 37.)

Num Mix(Num a, Num b) => a ^ b;

// To prune the secret number, calculate the value of the secret number modulo
// 16777216. Then, the secret number becomes the result of that operation. (If the
// secret number is 100000000 and you were to prune the secret number, the secret
// number would become 16113920.)

Num Prune(Num a) => a % 16777216; // ie take first 24 digits


In [6]:
// In particular, each buyer's secret number evolves into the next secret number in the sequence via the following process:

Num Evolve(Num secret)
{
    // Note: this function will overflow a 32-bit integer. Technically it's OK
    // since the bits are lost from the top and the result is pruned to the 24
    // lower bits

    // checked {

        // Each step of the above process involves mixing and pruning:

        // Calculate the result of multiplying the secret number by 64. Then, mix
        // this result into the secret number. Finally, prune the secret number.
        secret = Prune(Mix(secret * 64, secret));

        // Calculate the result of dividing the secret number by 32. Round the
        // result down to the nearest integer. Then, mix this result into the secret
        // number. Finally, prune the secret number.
        secret = Prune(Mix(secret / 32, secret));

        // Calculate the result of multiplying the secret number by 2048. Then, mix
        // this result into the secret number. Finally, prune the secret number.

        secret = Prune(Mix(secret * 2048, secret));
    
    // }

    return secret;
}

In [7]:
// So, if a buyer had a secret number of 123, that buyer's next ten secret numbers would be:

// 15887950
// 16495136
// 527345
// 704524
// 1553684
// 12683156
// 11100544
// 12249484
// 7753432
// 5908254

Num testSecret = 123;
foreach (var s in Enumerable.Range(0, 10))
{
    testSecret = Evolve(testSecret);
    Console.WriteLine(testSecret);
}

15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254


In [8]:
Num Evolve2K(Num secret) => Enumerable.Range(0, 2000).Aggregate(secret, (s, i) => Evolve(s));

long EvolveSum(string[] inputLines) => inputLines.Select(Num.Parse)
                                        .Select(Evolve2K)
                                        .Sum(s => (long)s);

In [9]:
// For each buyer, simulate the creation of 2000 new secret numbers. What is the sum of the 2000th secret number generated by each buyer?

var part1Answer = EvolveSum(inputLines);
Console.WriteLine(part1Answer);

13753970725


In [10]:
// 13753970725 is correct
Ensure(13753970725, part1Answer);